# Run chromVAR on example data

In [1]:
suppressMessages(library(chromVAR))
suppressMessages(library(Repitools))
suppressMessages(library(SummarizedExperiment))

In [2]:
data(example_counts, package = "chromVAR")